In [164]:
import pickle, json

import discovery as ds

import numpy as np
import jax
import jax.numpy as jnp

import matplotlib.pyplot as pp

In [198]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

Load pulsars

In [166]:
psrfiles = !ls ../data
psrs = [ds.Pulsar.read_feather(f'../data/{psrfile}') for psrfile in psrfiles[:2]]
psrs

Standard model 3a

In [182]:
Tspan = ds.getspan(psrs)

gbl = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                           for psr in psrs),
                          ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw'))

logl = gbl.logL

In [183]:
p0 = ds.sample_uniform(logl.params)
p0

{'B1855+09_rednoise_gamma': 6.337026790487537,
 'B1855+09_rednoise_log10_A': -17.100943473787986,
 'B1937+21_rednoise_gamma': 0.4449595245917949,
 'B1937+21_rednoise_log10_A': -12.859995213958111,
 'gw_log10_A': -12.968476457039607,
 'gw_gamma': 5.870155173424073}

In [184]:
logl(p0)

Array(429841.62961266, dtype=float64)

Global GP with multiple components (different spectra and different ORFs)

In [185]:
Tspan = ds.getspan(psrs)

gbl2 = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                            for psr in psrs),
                           ds.makegp_fourier_global(psrs,
                                                    [ds.powerlaw, ds.powerlaw],
                                                    [ds.hd_orf, ds.dipole_orf],
                                                    14, T=Tspan, name='gw'))

logl2 = gbl2.logL

Note the parameter names follow from the orf functions

In [188]:
p2 = ds.sample_uniform(logl2.params)
p2

{'B1855+09_rednoise_gamma': 1.3079309930662648,
 'B1855+09_rednoise_log10_A': -11.5753220526957,
 'B1937+21_rednoise_gamma': 4.803336230638544,
 'B1937+21_rednoise_log10_A': -11.4686576928151,
 'gw_dipoleorf_gamma': 4.14579350147083,
 'gw_dipoleorf_log10_A': -16.360764555101827,
 'gw_hdorf_gamma': 0.409405763199836,
 'gw_hdorf_log10_A': -17.066065343608827}

In [189]:
logl2(p2)

Array(440901.76079345, dtype=float64)

Conditional distribution

In [203]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

In [227]:
psrfiles = !ls ../data
psrs = [ds.Pulsar.read_feather(f'../data/{psrfile}') for psrfile in psrfiles[:3]]
psrs

In [304]:
Tspan = ds.getspan(psrs)

outpsr = 0
outnoise = ds.makenoise_measurement_simple(psrs[0], {f'{psrs[outpsr].name}_efac': 1.0,
                                                     f'{psrs[outpsr].name}_log10_t2equad': 0.0})

gbl = ds.GlobalLikelihood([ds.PulsarLikelihood([psr.residuals,
                                                outnoise if ipsr == outpsr else ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                           for ipsr, psr in enumerate(psrs)],
                          ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw'))

cond = gbl.conditional

In [255]:
cond.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_log10_A',
 'gw_gamma']

In [305]:
p0 = ds.sample_uniform(cond.params)

In [306]:
cdist = cond(p0)

In [307]:
cdist[0][:28]

Array([ 1.12069241e-06,  2.24387097e-06,  5.03134828e-07,  4.72657644e-07,
        3.03832707e-08, -2.85461849e-07, -1.19475192e-07, -3.74678655e-08,
        9.22343885e-09,  1.48455111e-08,  3.16224204e-08,  2.97032414e-08,
       -1.18850595e-08, -3.88159253e-08, -2.48324536e-08, -1.52352045e-08,
        5.97552610e-09,  1.17576269e-08, -5.15913907e-09,  7.93599810e-09,
       -1.26668586e-08,  2.07898044e-09, -2.70123638e-09,  5.89471446e-10,
        2.90228065e-09, -2.98293031e-10,  1.74988565e-09,  9.49270808e-09],      dtype=float64)

In [308]:
cdist[0][28:56]

Array([ 3.37941189e-06,  6.46632942e-06,  1.41167808e-06,  1.26962719e-06,
        1.01725483e-07, -8.09321786e-07, -3.13519679e-07, -1.53921103e-07,
        6.17988269e-09,  7.91679329e-09,  6.87678104e-08,  6.19571912e-08,
       -3.26126999e-08, -1.02200056e-07, -6.24997994e-08, -3.21836545e-08,
        1.68328339e-08,  3.65409535e-08, -9.83934993e-09,  2.18050965e-08,
       -2.73277544e-08,  4.84771461e-09, -2.50718732e-09,  6.16639810e-10,
        9.77029876e-09, -1.72231799e-09,  4.54428210e-09,  2.43176752e-08],      dtype=float64)

Check that the prediction is insensitive to outpsr, but not to the others.

In [309]:
p1 = {**p0, f'{psrs[outpsr].name}_rednoise_gamma': 4.0, f'{psrs[outpsr].name}_rednoise_log10_A': -15.0}

In [310]:
cond(p1)[0][:28] - cdist[0][:28]

Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)

In [311]:
p2 = {**p0, f'{psrs[outpsr+1].name}_rednoise_gamma': 4.0, f'{psrs[outpsr+1].name}_rednoise_log10_A': -15.0}

In [312]:
cond(p2)[0][:28] - cdist[0][:28]

Array([-8.48529036e-07, -1.59088736e-06, -1.75784722e-07, -1.18920997e-07,
       -1.76856128e-08,  4.46703699e-08,  3.58167822e-09,  7.46686256e-09,
        1.18562761e-09, -1.13984642e-09, -7.96329290e-09, -1.96057131e-09,
        1.06596039e-10,  4.40796576e-09,  6.71563635e-10, -3.28433243e-09,
       -2.62423806e-09,  2.42834676e-10,  9.46841944e-10,  4.03087931e-10,
       -7.73259402e-10, -2.36556195e-09, -1.20849575e-09,  1.16142875e-09,
        1.21897854e-09,  1.44844421e-10, -5.63799296e-10, -1.12631805e-09],      dtype=float64)

The second term in the conditional is `cf = L^-1` where S = L L^T is the variance of the normal distribution.

In [370]:
outrange  = slice(outpsr*28, (outpsr+1)*28)

mean      = cdist[0][outrange]
righthand = jnp.identity(len(cdist[0]))[:, outrange]
sigmalow  = ds.matrix.jsp.linalg.cho_solve(cdist[1], righthand)
variance  = sigmalow[outrange, :]

Check that the mean matches. Note this requires that a list be fed to ds.GlobalLikelihood above, but it can be safely changed to a generator in production.

In [331]:
ksolves = [psl.N.make_kernelsolve(psl.y, F) for psl, F in zip(gbl.psls, gbl.globalgp.Fs)]
FtNmy = ds.matrix.jnp.concatenate([ksolve(p0)[0] for ksolve in ksolves])
ds.matrix.jnp.dot(sigmalow.T, FtNmy) - cdist[0][:28]

Array([ 6.35274710e-22,  4.23516474e-22, -1.05879118e-22, -2.11758237e-22,
        8.60267837e-23,  1.58818678e-22,  0.00000000e+00,  2.64697796e-23,
        6.45200878e-23,  1.02570396e-22, -2.91167576e-22, -8.60267837e-23,
        2.87858853e-22,  3.30872245e-23, -2.97785021e-23, -1.05879118e-22,
       -2.48154184e-23,  7.61006164e-23,  2.23338765e-23, -2.48154184e-23,
       -2.15066959e-23,  1.19941189e-23, -6.98967618e-23,  7.23783036e-24,
       -1.81979735e-23,  1.31314922e-23,  4.96308368e-24, -1.65436123e-24],      dtype=float64)

In [393]:
def makegp_fourier_delay(psr, components, T=None, name='fourierGP'):
    argname = f'{psr.name}_{name}_delay'
    
    f, df, fmat = ds.fourierbasis(psr, components, T)
    Fmat = ds.matrix.jnparray(fmat)
    
    def delayfunc(params):
        return ds.matrix.jnp.dot(fmat, params[argname])
    delayfunc.params = [argname]

    return delayfunc

ds.makegp_fourier_delay = makegp_fourier_delay

In [394]:
def gpvariance(f, df, variance):
    return variance

In [484]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

In [485]:
psr = psrs[outpsr]
spl = ds.PulsarLikelihood([psr.residuals,
                           ds.makenoise_measurement(psr, psr.noisedict),
                           ds.makegp_ecorr(psr, psr.noisedict),
                           ds.makegp_timing(psr, variance=1e-14),
                           ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise'),
                           ds.makegp_fourier_delay(psr, 14, name='gw'),
                           ds.makegp_fourier(psr, gpvariance, 14, T=Tspan, priordim=2, name='gw')
                           ])

logl = spl.logL

In [486]:
logl.params

['B1855+09_gw_delay',
 'B1855+09_gw_variance',
 'B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A']

Check

In [511]:
Nmat = spl.N.P_var.getN({**pout, 'B1855+09_gw_delay': mean, 'B1855+09_gw_variance': variance})
print(np.sum(Nmat[:60,:60] - np.diag(np.diag(Nmat[:60,:60]))))
print(Nmat[60:,60:])

0.0
[[ 1.31494579e-14  4.00657769e-18 -2.98759762e-17 -2.79209532e-17
   1.29008727e-18  1.01109141e-17  6.00787470e-19  5.59450723e-18
  -4.55038036e-19  1.08262617e-18  6.30991243e-18 -3.21841822e-18
  -3.23834052e-18 -3.51155943e-18 -4.35746231e-18  4.11263501e-18
   3.35445373e-18 -1.81548818e-19 -5.77642544e-19 -2.96220468e-18
  -3.03322602e-19  3.08492645e-19  3.76698741e-19  3.75129948e-19
  -7.41820667e-19  4.88516686e-19  2.62218092e-19 -2.99060222e-20]
 [ 4.00657769e-18  1.31587101e-14  1.86841513e-17 -1.57723352e-17
   2.75108707e-18 -1.74774799e-18  1.69474047e-18  7.00973434e-18
   9.77371427e-18 -2.37697012e-18  1.39459387e-19 -2.71489306e-18
   4.44432547e-18  2.16579908e-18  2.26171643e-18 -3.68347619e-18
  -2.79487602e-20  2.26300804e-19  9.55471234e-19  1.73020885e-19
  -1.00805657e-18 -1.73573487e-18 -1.66181940e-18  1.11403682e-18
   8.51063217e-19  3.69204209e-19 -3.75578525e-19 -3.59951012e-19]
 [-2.98759762e-17  1.86841513e-17  3.59608561e-15  8.36058937e-18
  -1

In [516]:
pout = ds.sample_uniform([p for p in logl.params if 'gw' not in p])

In [519]:
logl({**pout, 'B1855+09_gw_delay': mean, 'B1855+09_gw_variance': variance})

Array(100143.63757598, dtype=float64)